In [9]:
# =============================================================================
# 2 Hick-Hyman Law
# ------------------------
# human performance principle depicting choice-reaction time
# GUI Usage:
# =============================================================================

import tkinter as tk
from tkinter import messagebox
import matplotlib.pyplot as plt
# import numpy as np
import random
import time as tm
from random import randint

def defineStimuliCollection():
    stimuliBasePositionX = 2
    stimuliBasePositionY = 2
    stimuliSideLength = 30
    stimuliFontSize = 20
    global stimuliKey
    stimuliKey = ['A', 'E', 'R', 'G', 'H', 'U', 'I', 'L']
    global stimuli
    stimuli = [0 for x in range(8)]
    global stimuliText
    stimuliText = [0 for x in range(8)]
    for x in range(8):
        stimuli[x] = frameGameArea.create_rectangle(stimuliBasePositionX, stimuliBasePositionY + x*32, stimuliBasePositionX + stimuliSideLength, stimuliBasePositionY + stimuliSideLength + x*32, fill='#C0FFF8', activefill='#63B2A9', tags=stimuliKey[x], state=tk.HIDDEN)
        stimuliText[x] = frameGameArea.create_text(stimuliBasePositionX + 0.5*stimuliSideLength, stimuliBasePositionY + 0.5*stimuliSideLength + x*32, text=stimuliKey[x], font=stimuliFontSize, tags=stimuliKey[x], state=tk.HIDDEN)
        root.bind("<KeyPress>", onKeyHit)

# get required amount of stimuli set
def generateStimuliSet(argAmount):
    global indexList
    for i in range(argAmount):
        indexList.append(i)
    print (indexList)

# in experiment 1
def generateRandomStimuli(argIndexList):
    # generate 1 random stimulus from the stimuli set
    global indexCurrent
    # indexCurrent is a list
    indexCurrent = random.sample(argIndexList, 1)
    frameGameArea.itemconfigure(stimuli[indexCurrent[0]], state=tk.NORMAL)
    frameGameArea.itemconfigure(stimuliText[indexCurrent[0]], state=tk.NORMAL)
    print (indexCurrent)
    # record timestamp when generating ths stimuli
    global timeStart
    timeStart.append(tm.clock())
    print (timeStart)

        # animate stimuli
        #  root.after(20, animateStimuli(indexList, argSpeed))
        #             frameGameArea.move(stimuli[index], argSpeed, 0)
        #             frameGameArea.move(stimuliText[index], argSpeed, 0)

def onKeyHit(event):
    # check which key is hit and delete associated object
    def hitRightKey():
        # get which key is hit
        letterHit = event.char
        if letterHit == stimuliKey[indexCurrent[0]] or letterHit == stimuliKey[indexCurrent[0]].lower():
            indexHit.append(indexCurrent[0])
            frameGameArea.itemconfigure(stimuliKey[indexCurrent[0]], state=tk.HIDDEN)
            global timeHit
            timeHit.append(tm.clock())
            print (timeHit)
            print ("--------------")
            # print (indexHit)
        # stimuli set use 5 times per set
        global counter
        if counter < 4:
            generateRandomStimuli(indexList)
            counter += 1
        # global flagFindKeyFinished
        # if flagFindKeyFinished is False:
        #     # get which key is hit
        #     letterHit = event.char
        #     global indexHit
        #     for index in indexList:
        #         if letterHit == stimuliKey[index] or letterHit == stimuliKey[index].lower():
        #             indexHit.append(index)
        #             frameGameArea.itemconfigure(stimuliKey[index], state=tk.HIDDEN)
        #             print (indexHit)
        #     # if finish hitting all stimuli key
        #     if sorted(indexHit) == sorted(indexCurrent):
        #         flagFindKeyFinished = True
        #         global counter
        #         if counter <= 5:
        #             experiment1()
        #             counter += 1
    def recordReactionTime():
        global timeHit
        global timeStart
        timeDifference = (timeHit[counter-1] - timeStart[counter-1]) # in millisecond
        global timeDelta
        timeDelta.append(timeDifference)
        print (timeDelta)
    # def recordError():
    hitRightKey()
    recordReactionTime()


# experiment 1: require accuracy, vary the amount of alternatives, adjust flying speed
# Reaction Time ~ f(amount of alternatives)
def experiment1():
    # global flagFindKeyFinished
    # flagFindKeyFinished = False
    # calculate speed from amountOfStimuli
    global timeStart
    timeStart = []
    global timeHit
    timeHit = []
    global timeDelta
    timeDelta = []
    generateStimuliSet(amountOfStimuli)
    # display what current random stimuli set is
    generateRandomStimuli(indexList)

# experiment 2: require speed, 8 alternatives, high flying speed
# Reaction Time ~ f(error rate)
def experiment2():
    print ("exp 2")

# def feedback():
#
# def modellingHH():

def onHelpClicked():
    tk.messagebox.showinfo("Usage Help", "blah blah...")

def onNextClicked():
    amountOfStimuli += 1
    if amountOfStimuli <= 8:
        print ("Next Round!")
        indexHit = []
        global flagFindKeyFinished
        flagFindKeyFinished = False
        # calculate speed from amountOfStimuli
        generateStimuli(amountOfStimuli, 30)

# GUI Layout
root = tk.Tk()
root.title("Hick-Hyman Law")
root.resizable(width=False, height=False)
frameExplain = tk.Frame(root)
global gameAreaHeight
gameAreaHeight = 300
frameGameArea = tk.Canvas(root, bd=2, bg='white', relief=tk.GROOVE, width=500, height=gameAreaHeight)
frameBonus = tk.Frame(root)
frameExplain.grid(row=0, column=0, padx=10, pady=10)
frameGameArea.grid(row=1, column=0, padx=10, pady=10)
frameGameArea.grid_propagate(0)
frameBonus.grid(row=0, column=1, padx=10, pady=10)
buttonHelp = tk.Button(frameExplain, text="Usage Help", command=onHelpClicked)
buttonStart = tk.Button(frameExplain, text="Next", command=onNextClicked)
labelPause = tk.Label(frameExplain, text="Press <space> key to pause at any time")
labelRequirement = tk.Label(frameBonus, text="Requirement here")
labelFeedback = tk.Label(frameBonus, text="chronoscope or hitting points here", bd=2, relief=tk.RAISED)
buttonHelp.grid(row=0, column=0, ipadx=10, ipady=10, padx=10, sticky=tk.W)
buttonStart.grid(row=0, column=1, ipadx=10, ipady=10, padx=10, sticky=tk.W)
labelPause.grid(row=0, column=2, padx=10, sticky=tk.W)
labelRequirement.grid(row=0, column=0, padx=10, pady=10)
labelFeedback.grid(row=1, column=0, ipadx=10, ipady=10, padx=10, pady=10)

# initialization
defineStimuliCollection()
global counter
counter = 0
global flagFindKeyFinished
flagFindKeyFinished = False
global indexList
indexList = []
global indexHit
indexHit = []
global amountOfStimuli
amountOfStimuli = 2
experiment1()
root.mainloop()
# while counter <=5:
#     if flagFindKeyFinished is True:
#         flagFindKeyFinished = False
#         counter += 1
#         experiment1()


# empirical data plotting

# result interpretation


[0, 1]
[1]
[1028.4780719805085]
[1030.422081708517]
--------------
[1]
[1028.4780719805085, 1030.4230910427475]
[1.9440097280084956]
[1030.422081708517, 1031.3125985000875]
--------------
[1]
[1028.4780719805085, 1030.4230910427475, 1031.3135340564747]
[1.9440097280084956, 0.8895074573399597]
[1030.422081708517, 1031.3125985000875, 1032.0051533379142]
--------------
[0]
[1028.4780719805085, 1030.4230910427475, 1031.3135340564747, 1032.0061080054293]
[1.9440097280084956, 0.8895074573399597, 0.6916192814394435]
[1030.422081708517, 1031.3125985000875, 1032.0051533379142, 1032.5481227094424]
--------------
[1]
[1028.4780719805085, 1030.4230910427475, 1031.3135340564747, 1032.0061080054293, 1032.5490835991855]
[1.9440097280084956, 0.8895074573399597, 0.6916192814394435, 0.5420147040131269]
[1030.422081708517, 1031.3125985000875, 1032.0051533379142, 1032.5481227094424, 1033.0685027275579]
--------------
[1.9440097280084956, 0.8895074573399597, 0.6916192814394435, 0.5420147040131269, 0.542014

In [ ]:
random.shuffle(indexList)
indexList